In [1]:
from pathlib import Path
import flopy
import pandas as pd
from simple_modflow.modflow.mf6.voronoiplus import VoronoiGridPlus as Vor
from simple_modflow.modflow.mf6.voronoiplus import TriangleGrid as Triangle
from simple_modflow.modflow.mf6.boundaries import Boundaries
import numpy as np
from shapely import Polygon
from simple_modflow.modflow.mf6.headsplus import HeadsPlus as hp
import shapely as shp
from simple_modflow.modflow.mf6 import mfsimbase as mf
from simple_modflow.modflow.mf6.recharge import RechargeFromShp as Rch
import itertools
import pickle
from simple_modflow.modflow.mf6.mfsimbase import SimulationBase
from simple_modflow.modflow.calcs.cj_approximation import CooperJacob as CJ
import figs as f
from simple_modflow.modflow.utils.surfaces import InterpolatedSurface as S
from pandas import IndexSlice as idxx

model_path = Path(r"C:\Users\lukem\mf6\LkPt_v6\LkPt_v6.model")
vor_path = Path(r'C:\Users\lukem\Python\MODFLOW\LakePointe\new_vor_lakepointe.vor')
with open(vor_path, 'rb') as file:
    vor: Vor = pickle.load(file)
with open(model_path, 'rb') as file:
    model: SimulationBase = pickle.load(file)

C:\Users\lukem\AppData\Local\Programs\Python\Python312\Lib\site-packages\reportlab\lib\rl_safe_eval.py:12: DeprecationWarning: ast.NameConstant is deprecated and will be removed in Python 3.14; use ast.Constant instead
  haveNameConstant = hasattr(ast,'NameConstant')


In [7]:
surf = S(vor=vor, model=model, kstpkper=(39, 0), layer=0)
surf.plot()

In [8]:
hds = hp(hds_path=model.model_output_folder_path.joinpath(f'{model.name}.hds'), vor=vor)
layer_nums = vor.gdf_topbtm.columns[1:].to_list()
hover = {layer: vor.gdf_topbtm.loc[:, layer].to_list() for layer in layer_nums}
hds.plot_choropleth((39, 6), zoom=13, plot_mounding=True, custom_hover=hover, layer=0, zmax=5, zmin=0).show()

In [2]:
start_exponent = 0  # Starting exponent (10^1)
end_exponent = 0.6    # Ending exponent (10^3)
num_points = 5     # Number of points in the range

# Generate the logarithmic range
log_range = np.logspace(start_exponent, end_exponent, num=num_points)

times = (pd.Series(log_range)-1).tolist()

In [3]:
times

[0.0,
 0.4125375446227544,
 0.9952623149688795,
 1.8183829312644537,
 2.9810717055349722]

In [10]:
qs = [100 for q in range(3)]
ks = [20, 200, 2000]
bs = [50 for q in range(3)]
Ss = [0.3 for q in range(3)]
rs = [100 for q in range(3)]
CJ().get_ds(ks=ks, bs=bs, Ss=Ss, rs=rs, times=times, qs=qs)

[[-inf,
  -4.138601337179834,
  -1.0316639161271406,
  1.0945821026212361,
  2.838541615544204],
 [-inf,
  0.3984661330625654,
  0.7091598751678346,
  0.9217844770426724,
  1.0961804283349692],
 [-inf,
  0.12107923998431144,
  0.15214861419483836,
  0.17341107438232212,
  0.19085066951155183]]

In [22]:
CJ().cooper_jacob_drawdown(q=400_000, S=0.2, t=20, r=100, k=200, b=20)

82.46662452346483

In [6]:
l = []
for t in range(8):
    l += [t]
l

[0, 1, 2, 3, 4, 5, 6, 7]

In [ ]:
recharge = Path(r"C:\Users\lukem\Python\MODFLOW\LakePointe\inputs\shp\recharge\v2_with_fill\LP_RechargeExJoined_ply_20250529.shp")
uid = 'UID'

rch = Rch(model, vor, recharge, uid, rch_fields=slice('OCT_ftpd','Annual_ftp'), rch_fields_to_pers=[12 for x in range(33)])
rch.nper = 125

In [ ]:
rch.get_rch()[2]

In [ ]:
("""Example simple model"""
tri = Triangle(
    model_ws=Path.cwd(),
    angle=30,
    #maximum_area=50
)
tri.add_rectangle(x_dist=500, y_dist=500, origin=(-200, -200))

coords = ((-100., -100.), (-100., 150.), (150., 150.), (150., -100.), (-100., -100.))
polygon = shp.Polygon(coords)
tri.add_polygon(densify_poly(polygon, distance_between=1))
tri.add_rectangle(max_area=10)
# tri.add_polygon(densify_poly(polygon, distance_between=0.1).buffer(-2))
tri.model_ws = Path.cwd().joinpath('sample_model_output')
tri.build()
vor = Vor(tri=tri)
vor.plot_choropleth().show()

In [ ]:

# get bottom elevs
strike = 30  # Strike given in degrees from north
dip = 20  # Dip given in degrees from horizontal
known_point = (50, 200, 0)  # Known point (x, y, elevation)
pixel_size = 1  # Pixel size
bottom_raster_path = Path.cwd().joinpath('sample_model_output', 'bottom_raster.tif')
top_raster_path = Path.cwd().joinpath('sample_model_output', 'top_raster.tif')
vor = Vor(tri)

In [ ]:
fig = vor.plot_choropleth()
fig.show()

In [ ]:


bottom_elevs = vor.get_raster_from_strike_dip(strike, dip, known_point, pixel_size, bottom_raster_path)
top_elevs = vor.get_raster_from_strike_dip(0, 0, (0,0,50), 1, top_raster_path)
vor = Vor(tri, rasters=[bottom_raster_path, top_raster_path])
nper = 31
center_cells = [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
                57, 58, 59, 60, 61, 62, 63, 98, 123, 124, 163, 164, 165, 193, 194, 195, 196, 210, 211, 212, 213,
                234, 235, 236, 314, 337, 374, 375, 388, 392]
rch_trans = [np.random.random() + 4 for per in range(nper)]
rch_dict = {}
for per in range(nper):
    cell_list = []
    for cell in range(vor.ncpl):
        if cell in center_cells:
            cell_list.append([cell, rch_trans[per]])
        else:
            cell_list.append([cell, 0.01])

    rch_dict[per] = cell_list
model = SimpleModel(
    vor,
    k=5,
    #bottom=bottom_elevs['elev'].to_list(),
    #top=50,
    nper=nper,
    rch_dict=rch_dict
)
model.run_simulation()
hds = hp(hds_path=model.model_output_folder_path.joinpath('mf6_model.hds'), vor=vor)
hds.plot_choropleth((19, 0), zoom=19, plot_mounding=True).show()

In [ ]:
import plotly.graph_objs as go
import rasterio
from pathlib import Path
import rioxarray, xarray

raster_path = Path(r'C:\Users\lukem\Python\MODFLOW\LakePointe\inputs\surfaces\raster_surface\finals')
rast_dict = {}
for file in Path.iterdir(raster_path):
    if file.suffix == '.tif' or file.suffix == '.tiff':
        raster = rioxarray.open_rasterio(file)
        rast_dict[file.stem] = raster


In [ ]:
import pandas as pd
fig = go.Figure()
for key in rast_dict.keys():
    rast = rast_dict[key]
    df_rast = pd.DataFrame(rast.values[0])
    df_rast = df_rast[df_rast>0]
    fig.add_surface(z=df_rast, x=rast.x, y=rast.y, name=key, showlegend=True, scene='scene')
surface_scene = go.layout.Scene(zaxis_range=[0, 1000])
fig.update_scenes(patch=surface_scene)

In [ ]:
fig.show(renderer='browser')